### T&D 3 - Limpeza de Dados
Autor: Felipe Reis


###Import de dependências e tabelas

In [ ]:
!pip install pyspark==3.5.0

In [ ]:
!pip install pandas

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import pandas as pd

sc = SparkSession.builder.master('local[*]').getOrCreate()

In [ ]:
df = pd.read_csv('sample_data/salario_dataset.csv')
spark_df = sc.createDataFrame(df)

###Limpeza dos dados categóricos



In [ ]:
cleaned_spark_df = (
    spark_df
    .withColumn('empresa', F.when(F.col('empresa').like('lnnNSf5YDt'), 'Outra').otherwise(F.col('empresa')))
    .withColumn('cargo', F.when(F.col('cargo').like('s2iOR6wzbFZtK5T'), 'Outro').otherwise(F.col('cargo')))
    .withColumn('nome', F.when(F.col('nome').like('NaN'), 'Outro').otherwise(F.col('nome')))
    .withColumn('sobrenome', F.when(F.col('sobrenome').like('NaN'), 'Outro').otherwise(F.col('sobrenome')))
    )

###Limpeza de dados numéricos

In [ ]:
cleaned_spark_df = (
    cleaned_spark_df
    .withColumn(
      "anos_de_experiencia",
      F.when(F.col("anos_de_experiencia") < 0, None).otherwise(F.col("anos_de_experiencia"))
    )
    .withColumn(
      "salario",
      F.when((F.col("salario") < 0) | (F.col("salario") == "NaN"), None).otherwise(F.col("salario"))
    ))

median_anos = cleaned_spark_df.approxQuantile("anos_de_experiencia", [0.5], 0.01)[0]
cleaned_spark_df = cleaned_spark_df.fillna({"anos_de_experiencia": median_anos})

cleaned_spark_df.show()

In [ ]:
quantis = (
    spark_df
    .filter(F.col('salario') != 'NaN')
    .approxQuantile('salario', [0.25, 0.75], 0.01)
    )

Q1, Q3 = quantis[0], quantis[1]

IQR = Q3 - Q1

limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

salario_filtrado = (spark_df.filter((F.col("salario") >= limite_inferior) & (F.col("salario") <= limite_superior)))
median_salario = salario_filtrado.approxQuantile("salario", [0.5], 0.01)[0]

cleaned_spark_df = cleaned_spark_df.fillna({"salario": median_salario})